# Imports

In [1]:
import os
import pandas as pd
import time

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tqdm import tqdm

import ollama

In [2]:
# Force CUDA usage
os.environ["OLLAMA_BACKEND"] = "cuda"
os.environ["OLLAMA_NUM_THREADS"] = "16"

### Parameters

In [3]:
# List of models
ml_models = [
    ('Logistic Regression', LogisticRegression(random_state=42, max_iter=1000)), # 2 sec
    ('Random Forest', RandomForestClassifier(random_state=42)), # 2 min
    ('SVM', SVC(random_state=42)), # 30 min
    ('KNN', KNeighborsClassifier()), # 30 sec
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)) # 30 sec
]

In [4]:
# LLMs
models = ['llama3.2:1b', 'llama3.2:3b', 'llama3.1']

In [5]:
# Vectorize (Bag of words representation)
vectorizer = CountVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))

In [6]:
# Number of rows that LLM will generate and compare to
n_rows = 3000

# Load data

In [7]:
# Load the CSV file into a pandas DataFrame
base_df = pd.read_csv('IMB_preprocessed_2025_04_06.csv')

# We want to be able to read the full reviews
pd.set_option('display.max_colwidth', None) 

# Display the first 5 records
base_df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.\nThe first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.\nIt is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.\nI would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",1
1,"A wonderful little production. \nThe filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. \nThe actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. \nThe realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.\nThis was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.\nThis may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",1
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.\nThis movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.\nOK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a dram

# Experimential setup

In [8]:
def experimental_setup(df):
    # Split into X and y
    X = df['review']
    y = df['sentiment']
    
    # Split data into train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Only train on the first n_rows
    X_train = X_train[:n_rows]
    y_train = y_train[:n_rows]

    # Transform text reviews to Bag of Words representation
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    # Print the shapes of the sets
    print(f"Training Set: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
    print(f"Test Set: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}")
    print()

    return X_train, y_train, X_test, y_test

# Modeling and Performance metrics

In [9]:
# Function to train and evaluate models with multiple metrics
def evaluate_models_with_metrics(models, X_train, y_train, X_test, y_test):
    results = []

    for name, model in models:      
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on test set
        y_test_pred = model.predict(X_test)

        # Store results for the model
        model_results = {
            'Model': name,
            'Test Accuracy': accuracy_score(y_test, y_test_pred),
            'Test Precision': precision_score(y_test, y_test_pred),
            'Test Recall': recall_score(y_test, y_test_pred),
            'Test F1-Score': f1_score(y_test, y_test_pred)
        }
        
        results.append(model_results)

    # Convert results to a pandas DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [10]:
# Create the split
X_train_base, y_train_base, X_test_base, y_test_base = experimental_setup(base_df)

# Evaluate models with multiple metrics and print results
base_results = evaluate_models_with_metrics(ml_models, X_train_base, y_train_base, X_test_base, y_test_base)
display(base_results)

Training Set: X_train shape = (3000, 10000), y_train shape = (3000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)



,Model,Test Accuracy,Test Precision,Test Recall,Test F1-Score
0,Logistic Regression,0.8367,0.828764,0.851955,0.840200
1,Random Forest,0.8244,0.809659,0.851756,0.830174
2,SVM,0.8210,0.783855,0.890256,0.833674
3,KNN,0.5791,0.581309,0.588807,0.585034
4,Gradient Boosting,0.7952,0.761314,0.864656,0.809701


# Data Disambiguation

### Non-Qualitative loop

In [11]:
# Prompt
prompt = '''''Clarify any ambiguous phrases in the following review to make the meaning more precise, but do not alter the original meaning of the review. 
Return only the disambiguated text, do not return any explanations or additional commentary.

Review:
{review}'''

In [12]:
# Initialize a dictionary to store results per model
model_list = {}

# Important! You must only sample from the train set, you mustn't sample from the test set
# This is to prevent data leakage
X_train, _, y_train, _ = train_test_split(base_df['review'], base_df['sentiment'], test_size=0.2, random_state=42)
llm_train_df = pd.DataFrame({'review': X_train, 'sentiment': y_train})

# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")
    model_list[model] = []

    # Disambiguate n_rows
    for index, row in tqdm(llm_train_df[:n_rows].iterrows(), total=n_rows, desc="Disambiguating rows"):
        # Format the prompt baesd on the row
        formatted_prompt = prompt.format(review=row['review'])

        # Disambiguate
        response = ollama.generate(model=model, prompt=formatted_prompt)['response']

        # Store response and target variable
        model_list[model].append([response, row['sentiment']])

Processing Model: llama3.2:1b (Model 1/3)


Disambiguating rows: 100%|███████████████████████████████████████████████████████| 3000/3000 [3:56:58<00:00,  4.74s/it]


Processing Model: llama3.2:3b (Model 2/3)


Disambiguating rows: 100%|███████████████████████████████████████████████████████| 3000/3000 [7:10:17<00:00,  8.61s/it]


Processing Model: llama3.1 (Model 3/3)


Disambiguating rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [30:13:06<00:00, 36.26s/it]


# Comparison

In [13]:
def compare(base_df, llm_df):
    # Set 'Model' as the index in DataFrames
    base_df_results = base_df.set_index('Model')
    llm_df_results = llm_df.set_index('Model')
    
    # Rename evaluation metrics columns
    base_df_results = base_df_results.rename(columns={
        'Test Accuracy': 'Test Accuracy base', 'Test F1-Score': 'Test F1-Score base',
        'Test Precision': 'Test Precision base', 'Test Recall': 'Test Recall base'
    })

    llm_df_results = llm_df_results.rename(columns={
        'Test Accuracy': 'Test Accuracy llm altered', 'Test F1-Score': 'Test F1-Score llm altered',
        'Test Precision': 'Test Precision llm altered', 'Test Recall': 'Test Recall llm altered'
    })
    
    # Merge dfs by index
    raw_final_results = base_df_results.join(llm_df_results)

    # Show raw data results
    print("Raw performance metrics:")
    display(raw_final_results)
    
    # Calculate and display the difference matrix (% difference)
    diff_df = pd.DataFrame()
    diff_df['Test Accuracy % diff'] = round(((raw_final_results['Test Accuracy llm altered'] - raw_final_results['Test Accuracy base']) 
                                                   / raw_final_results['Test Accuracy base']) * 100, 1)
    diff_df['Test F1-Score % diff'] = round(((raw_final_results['Test F1-Score llm altered'] - raw_final_results['Test F1-Score base']) 
                                                 / raw_final_results['Test F1-Score base']) * 100, 1)
    diff_df['Test Precision % diff'] = round(((raw_final_results['Test Precision llm altered'] - raw_final_results['Test Precision base']) 
                                           / raw_final_results['Test Precision base']) * 100, 1)
    diff_df['Test Recall % diff'] = round(((raw_final_results['Test Recall llm altered'] - raw_final_results['Test Recall base']) 
                                           / raw_final_results['Test Recall base']) * 100, 1)
    print("% Difference df (between base and LLM altered df):")
    display(diff_df)
    print()

In [14]:
# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")

    # Get data and make a DataFrame
    disambiguation_df = pd.DataFrame(model_list[model], columns=['review', 'sentiment'])

    # Split into X and y
    X_train_disambiguation = disambiguation_df['review']
    y_train_disambiguation = disambiguation_df['sentiment']

    # Get original test set
    _, X_test_base, _, y_test_base = train_test_split(base_df['review'], base_df['sentiment'], test_size=0.2, random_state=42)
    
    # Transform text reviews to Bag of Words representation
    X_train_disambiguation = vectorizer.fit_transform(X_train_disambiguation)
    X_test_base = vectorizer.transform(X_test_base)

    # Print the shapes of the sets
    print(f"Training Set: X_train shape = {X_train_disambiguation.shape}, y_train shape = {y_train_disambiguation.shape}")
    print(f"Test Set: X_test shape = {X_test_base.shape}, y_test shape = {y_test_base.shape}")
    print()
    
    # Evaluate models with multiple metrics and display results
    results_disambiguation = evaluate_models_with_metrics(ml_models, X_train_disambiguation, y_train_disambiguation, X_test_base, y_test_base)
    
    # Compare the results between base and LLM altered df
    compare(base_results, results_disambiguation)

Processing Model: llama3.2:1b (Model 1/3)
Training Set: X_train shape = (3000, 10000), y_train shape = (3000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)

Raw performance metrics:


,Test Accuracy base,Test Precision base,Test Recall base,Test F1-Score base,Test Accuracy llm altered,Test Precision llm altered,Test Recall llm altered,Test F1-Score llm altered
Model,,,,,,,,
Logistic Regression,0.8367,0.828764,0.851955,0.840200,0.8260,0.818252,0.841635,0.829779
Random Forest,0.8244,0.809659,0.851756,0.830174,0.8193,0.799703,0.855725,0.826766
SVM,0.8210,0.783855,0.890256,0.833674,0.6605,0.601985,0.962889,0.740820
KNN,0.5791,0.581309,0.588807,0.585034,0.5292,0.607958,0.184957,0.283628
Gradient Boosting,0.7952,0.761314,0.864656,0.809701,0.7986,0.770138,0.855725,0.810679


% Difference df (between base and LLM altered df):


,Test Accuracy % diff,Test F1-Score % diff,Test Precision % diff,Test Recall % diff
Model,,,,
Logistic Regression,-1.3,-1.2,-1.3,-1.2
Random Forest,-0.6,-0.4,-1.2,0.5
SVM,-19.5,-11.1,-23.2,8.2
KNN,-8.6,-51.5,4.6,-68.6
Gradient Boosting,0.4,0.1,1.2,-1.0



Processing Model: llama3.2:3b (Model 2/3)
Training Set: X_train shape = (3000, 10000), y_train shape = (3000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)

Raw performance metrics:


...

,Test Accuracy % diff,Test F1-Score % diff,Test Precision % diff,Test Recall % diff
Model,,,,
Logistic Regression,0.1,0.2,-0.0,0.4
Random Forest,1.0,0.8,1.3,0.3
SVM,-0.1,-0.1,0.1,-0.4
KNN,-7.6,-38.1,0.8,-55.5
Gradient Boosting,0.2,0.1,0.5,-0.5



Processing Model: llama3.1 (Model 3/3)
Training Set: X_train shape = (3000, 10000), y_train shape = (3000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)

Raw performance metrics:


,Test Accuracy base,Test Precision base,Test Recall base,Test F1-Score base,Test Accuracy llm altered,Test Precision llm altered,Test Recall llm altered,Test F1-Score llm altered
Model,,,,,,,,
Logistic Regression,0.8367,0.828764,0.851955,0.840200,0.8347,0.819073,0.862473,0.840213
Random Forest,0.8244,0.809659,0.851756,0.830174,0.8277,0.812123,0.856122,0.833543
SVM,0.8210,0.783855,0.890256,0.833674,0.8202,0.787680,0.880532,0.831522
KNN,0.5791,0.581309,0.588807,0.585034,0.5206,0.607174,0.137726,0.224523
Gradient Boosting,0.7952,0.761314,0.864656,0.809701,0.7952,0.757268,0.873586,0.811279


% Difference df (between base and LLM altered df):


,Test Accuracy % diff,Test F1-Score % diff,Test Precision % diff,Test Recall % diff
Model,,,,
Logistic Regression,-0.2,0.0,-1.2,1.2
Random Forest,0.4,0.4,0.3,0.5
SVM,-0.1,-0.3,0.5,-1.1
KNN,-10.1,-61.6,4.4,-76.6
Gradient Boosting,0.0,0.2,-0.5,1.0
